In [5]:
import crocoddyl
import numpy as np
import example_robot_data
talos_arm= example_robot_data.load('talos_arm')
robot_model=talos_arm.model

In [6]:
# Defining a initial state
q0 = np.array([0.173046, 1., -0.52366, 0., 0., 0.1, -0.005])
x0 = np.concatenate([q0, np.zeros(talos_arm.model.nv)])


In [7]:
import pinocchio


class DifferentialFwdDynamics(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self, state, costModel):
        crocoddyl.DifferentialActionModelAbstract.__init__(self, state, state.nv, costModel.nr)
        self.costs = costModel
        self.enable_force = True
        self.armature = np.zeros(0)

    def calc(self, data, x, u=None):
        if u is None:
            u = self.unone
        q, v = x[:self.state.nq], x[-self.state.nv:]
        # Computing the dynamics using ABA or manually for armature case
        if self.enable_force:
            data.xout = pinocchio.aba(self.state.pinocchio, data.pinocchio, q, v, u)
        else:
            pinocchio.computeAllTerms(self.state.pinocchio, data.pinocchio, q, v)
            data.M = data.pinocchio.M
            if self.armature.size == self.state.nv:
                data.M[range(self.state.nv), range(self.state.nv)] += self.armature
            data.Minv = np.linalg.inv(data.M)
            data.xout = data.Minv * (u - data.pinocchio.nle)
        # Computing the cost value and residuals
        pinocchio.forwardKinematics(self.state.pinocchio, data.pinocchio, q, v)
        pinocchio.updateFramePlacements(self.state.pinocchio, data.pinocchio)
        self.costs.calc(data.costs, x, u)
        data.cost = data.costs.cost

    def calcDiff(self, data, x, u=None):
        q, v = x[:self.state.nq], x[-self.state.nv:]
        if u is None:
            u = self.unone
        if True:
            self.calc(data, x, u)
        # Computing the dynamics derivatives
        if self.enable_force:
            pinocchio.computeABADerivatives(self.state.pinocchio, data.pinocchio, q, v, u)
            data.Fx = np.hstack([data.pinocchio.ddq_dq, data.pinocchio.ddq_dv])
            data.Fu = data.pinocchio.Minv
        else:
            pinocchio.computeRNEADerivatives(self.state.pinocchio, data.pinocchio, q, v, data.xout)
            data.Fx = -np.hstack([data.Minv * data.pinocchio.dtau_dq, data.Minv * data.pinocchio.dtau_dv])
            data.Fu = data.Minv
        # Computing the cost derivatives
        self.costs.calcDiff(data.costs, x, u)

    def set_armature(self, armature):
        if armature.size is not self.state.nv:
            print('The armature dimension is wrong, we cannot set it.')
        else:
            self.enable_force = False
            self.armature = armature.T

    def createData(self):
        data = crocoddyl.DifferentialActionModelAbstract.createData(self)
        data.pinocchio = pinocchio.Data(self.state.pinocchio)
        data.multibody = crocoddyl.DataCollectorMultibody(data.pinocchio)
        data.costs = self.costs.createData(data.multibody)
        data.costs.shareMemory(data) # this allows us to share the memory of cost-terms of action model
        return data


In [8]:
# Create the cost functions
target = np.array([0.4, 0., .4])
state = crocoddyl.StateMultibody(robot_model)
frameTranslationResidual = crocoddyl.ResidualModelFrameTranslation(state,
                                            robot_model.getFrameId("gripper_left_joint"),
                                            target)
goalTrackingCost = crocoddyl.CostModelResidual(state, frameTranslationResidual)
xRegCost = crocoddyl.CostModelResidual(state, crocoddyl.ResidualModelState(state))
uRegCost = crocoddyl.CostModelResidual(state, crocoddyl.ResidualModelControl(state))

# Create cost model per each action model
runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)

# Then let's added the running and terminal cost functions
runningCostModel.addCost("gripperPose", goalTrackingCost, 1e2)
runningCostModel.addCost("stateReg", xRegCost, 1e-4)
runningCostModel.addCost("ctrlReg", uRegCost, 1e-7)
terminalCostModel.addCost("gripperPose", goalTrackingCost, 1e5)
terminalCostModel.addCost("stateReg", xRegCost, 1e-4)
terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7)

# Running and terminal action models
DT = 1e-3
actuationModel = crocoddyl.ActuationModelFull(state)
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModel), DT)
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModel), 0.)


In [10]:


#We create a trajectory with 250 knots

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 250
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel)



In [16]:
# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem)
log = crocoddyl.CallbackLogger()

# Using the meshcat displayer, you could enable gepetto viewer for nicer view
# display = crocoddyl.GepettoDisplay(talos_arm, 4, 4)
display = crocoddyl.MeshcatDisplay(talos_arm, 4, 4, False)
ddp.setCallbacks([log,
                  crocoddyl.CallbackVerbose(),
                  crocoddyl.CallbackDisplay(display)])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [17]:
# Emdebbed meshcat in this cell
display.robot.viewer.jupyter_cell()

In [18]:
# Solving it with the DDP algorithm
ddp.solve()

# Printing the reached position
frame_idx = talos_arm.model.getFrameId("gripper_left_joint")
xT = ddp.xs[-1]
qT = xT[:talos_arm.model.nq]
# \



iter     cost         stop         grad         xreg         ureg       step    ||ffeas||
   0  6.00288e+04  7.91470e-02  1.20714e+05  1.00000e-08  1.00000e-08  0.0078  1.00000e+00
   1  1.86695e+04  1.23794e-01  1.20048e+05  1.00000e-08  1.00000e-08  0.1250  0.00000e+00
   2  1.78169e+04  1.83329e+00  3.73176e+04  1.00000e-08  1.00000e-08  0.0312  0.00000e+00
   3  1.68568e+04  1.39142e+00  3.56111e+04  1.00000e-08  1.00000e-08  0.0625  0.00000e+00
   4  1.64638e+04  1.04344e+00  3.36694e+04  1.00000e-08  1.00000e-08  0.0156  0.00000e+00
   5  1.61065e+04  8.80911e-01  3.28675e+04  1.00000e-08  1.00000e-08  0.0156  0.00000e+00
   6  1.57519e+04  7.82130e-01  3.21353e+04  1.00000e-08  1.00000e-08  0.0312  0.00000e+00
   7  1.55007e+04  7.62760e-01  3.14227e+04  1.00000e-08  1.00000e-08  0.0156  0.00000e+00
   8  1.54268e+04  7.93924e-01  3.09232e+04  1.00000e-08  1.00000e-08  0.0156  0.00000e+00
   9  1.50970e+04  7.98627e-01  3.07918e+04  1.00000e-08  1.00000e-08  0.0312  0.00000e+00


  85  1.45294e+02  2.15397e-02  2.22739e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00
  86  1.41108e+02  2.03637e-02  2.07700e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00
  87  1.13974e+02  1.99193e-02  1.98222e+02  1.00000e-07  1.00000e-07  0.1250  0.00000e+00
  88  1.06520e+02  1.23865e-02  1.64270e+02  1.00000e-07  1.00000e-07  0.0625  0.00000e+00
  89  1.05463e+02  1.40823e-02  1.66343e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00
iter     cost         stop         grad         xreg         ureg       step    ||ffeas||
  90  1.04397e+02  1.50242e-02  1.65994e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00
  91  1.03597e+02  1.62696e-02  1.64874e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00
  92  1.02659e+02  1.75617e-02  1.64789e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00
  93  1.01809e+02  1.90199e-02  1.63762e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00
  94  1.00441e+02  2.05520e-02  1.62396e+02  1.00000e-07  1.00000e-07  0.0312  0.00000e+00


In [15]:
%matplotlib inline
# # Plotting the solution and the DDP convergence
crocoddyl.plotOCSolution(log.xs, log.us)
crocoddyl.plotConvergence(log.costs, log.u_regs, log.x_regs, log.grads, log.stops, log.steps)

# Visualizing the solution in gepetto-viewer
display.displayFromSolver(ddp)


AttributeError: module 'numpy' has no attribute 'asscalar'